# Motion Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import time
import socket
from urllib.request import urlopen
import json

# The multiprocessing Pool class is designed to create multiple processes
#from multiprocessing import Pool

# The multiprocessing.dummy module uses multiple threads for the pool, rather than multiple processes
from multiprocessing.dummy import Pool

LATEST_FIRMWARE = 3129

HOST_STATUS_UNKNOWN = 0
HOST_STATUS_READY = 1
HOST_STATUS_REFUSED = 2
HOST_STATUS_TIMEOUT = 3
HOST_STATUS_GAIERROR = 4

DEFAULT_HTTP_PORT=80

DEFAULT_PING_TIMEOUT=5
DEFAULT_JSON_TIMEOUT=10

PING_POOL_SIZE = 25
JSON_POOL_SIZE = 5

In [2]:
class Motion():
    '''Motion GPS'''
    
    def __init__(self, identifier):
        self.identifier = identifier

        self.hostname = f'motion-{identifier}'

        self.addrInfo = None
        self.status = HOST_STATUS_UNKNOWN

        self.json = {}
        
        
    def getAddrInfo(self, port=DEFAULT_HTTP_PORT):
        """Get address information (IP address) for Motion"""

        try:
            addrInfo = socket.getaddrinfo(self.hostname, port, proto=socket.IPPROTO_TCP)[0][4]

            # Sky SR101 router will sometimes return 10.10.10.10 and should be treated the same as socket.gaierror
            if addrInfo[0] == '10.10.10.10':
                raise socket.gaierror

        # Only really expect a socket.gaierror exception, either from socket.getaddrinfo or because of 10.10.10.10
        except socket.gaierror:
            self.addrInfo = None

        # Just in case of some other (unexpected) issue!
        except Exception:
            self.addrInfo = None

        # The IP address of the Motion has been successfully identified. Yay!
        else:
            self.addrInfo = addrInfo
            
        return self.addrInfo


    def getStatus(self, timeout=DEFAULT_PING_TIMEOUT):
        """Check the status of the Motion.

        Note the use of self.addrInfo instead of using the hostname with s.connect((host, port)).

        This avoids erroneous 'gaierror: [Errno -5] No address associated with hostname' under certain circumstances."""

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()

        # Status can only be checked if address information is available
        if self.addrInfo:
            try:
                # Attempt connection to the standard HTTP socket as determined by socket.getaddrinfo()
                s = socket.socket()
                s.settimeout(timeout)
                s.connect(self.addrInfo)

                # Close the connection in a timely fashion - good practice
                s.shutdown(socket.SHUT_RDWR)
                s.close()

            # This should only happen if the IP address does not belong to a Motion
            except ConnectionRefusedError:
                self.status = HOST_STATUS_REFUSED

            # This should only happen when the Motion has been disconnected from the WiFi
            except socket.timeout:
                self.status = HOST_STATUS_TIMEOUT

            # Just in case of some other (unexpected) issue!
            except Exception:
                self.status = HOST_STATUS_UNKNOWN

            # The Motion is accepting connections. Yay!
            else:
                self.status = HOST_STATUS_READY

        # Address information is not known
        else:
            self.status = HOST_STATUS_GAIERROR

        return self.status
   

    def fetchJson(self, path, port=DEFAULT_HTTP_PORT, timeout=DEFAULT_JSON_TIMEOUT):
        """Fetch JSON from Motion"""

        result = None

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()

        # JSON can only be fetched if address information is available
        if self.addrInfo:
            try:
                # Use addrInfo to avoid unnecessary DNS lookup which can take several seconds
                url = 'http://{}/{}'.format(self.addrInfo[0], path)

                # This is a simple but effective way to open a URL in Python
                response = urlopen(url, timeout=timeout)
                txt = response.read()

                # Parse the JSON and store it in a dictionary
                result = json.loads(txt)
                self.json[path] = result

            except Exception:
                print("Failed to retrieve {}".format(url))

        return result


    def summarise(self):
        """Print summary of device"""

        print(self.hostname)
        print()

        info = self.json['info.json']

        print('Username = {}'.format(info['motion']['username']))
        print('Identifier = {}'.format(info['motion']['identifier']))
        print()

        hardware = info['motion']['hardware']
        firmware = info['motion']['firmware']

        print('Hardware = {}'.format(hardware))
        print('Firmware = {}{}'.format(firmware, ' (latest)' if firmware == LATEST_FIRMWARE else ''))
        print()

        # 'filesystem level' always appears to be 3

        level = info['motion']['battery']['level']
        charging = info['motion']['battery']['charging']

        print('Battery = {}%'.format(level))
        print('Battery is {}charging'.format('not ' if charging == False else ''))
        print()

        settings = self.json['settings.json']

        print('GNSS rate = {}'.format(settings['settings']['gnss_rate']))
        print()

        # Note: 'units': {'speed': 'kn', 'distance': 'nm'} does not impact logs (mm/s and mm)
        #        'start_date' is in UTC
        #        'timezone' and 'units' are applied by the built-in webserver
        #        'mode' always appears to be 0

        #logs = fetchJson(addresses[i][0], 'logs.json', timeout=20)
        #for log in reversed(logs['logs'][-1:]):
            #print(log)
        #print()

## Test Code

Test lookups, etc

In [3]:
def getAddrInfo(motion, port=DEFAULT_HTTP_PORT):
    """Get address information for Motion"""
    
    return motion.getAddrInfo(port=port)


motions = []
for identifier in [619, 621, 633, 634]:
    motion = Motion(identifier)
    motions.append(motion)
    
pc1 = time.perf_counter()

# Process will usually take 10 seconds if any of the minis are not in DNS
# This is related to the OS and cannot be overriden by a timeout
pool = Pool(PING_POOL_SIZE)
addrInfos = pool.map(getAddrInfo, motions)
pool.close()
pool.join()

pc2 = time.perf_counter()
print("{} hosts looked up in {:0.2f} seconds".format(len(motions), pc2 - pc1))

4 hosts looked up in 3.11 seconds


In [4]:
def getStatus(motion, timeout=DEFAULT_PING_TIMEOUT):
    """Get status information for Motion"""
    
    return motion.getStatus(timeout=timeout)


knownMotions = []
for motion in motions:
    if motion.addrInfo:
        knownMotions.append(motion)

pc1 = time.perf_counter()

pool = Pool(PING_POOL_SIZE)
results = pool.map(getStatus, knownMotions)
pool.close()
pool.join()

pc2 = time.perf_counter()
print("{} hosts checked in {:0.2f} seconds".format(len(knownMotions), pc2 - pc1))
print()

for motion in knownMotions:
    if motion.status == HOST_STATUS_READY:
        print('{} is accepting requests'.format(motion.hostname))
    elif motion.status == HOST_STATUS_REFUSED:
        print('{} is refusing requests'.format(motion.hostname))
    elif motion.status == HOST_STATUS_TIMEOUT:
        print('{} is timing out'.format(motion.hostname))
    elif motion.status == HOST_STATUS_UNKNOWN:
        print('{} is in an unknown state'.format(motion.hostname))

4 hosts checked in 1.22 seconds

motion-619 is accepting requests
motion-621 is accepting requests
motion-633 is accepting requests
motion-634 is accepting requests


In [5]:
def fetchJson(motion, timeout=DEFAULT_JSON_TIMEOUT):
    """Get address information for host"""
    
    motion.fetchJson('info.json', timeout=timeout)
    motion.fetchJson('settings.json', timeout=timeout)

    return motion.json


pc1 = time.perf_counter()

onlineMotions = []
for motion in motions:
    if motion.status == HOST_STATUS_READY:
        onlineMotions.append(motion)

pc1 = time.perf_counter()

pool = Pool(JSON_POOL_SIZE)
results = pool.map(fetchJson, onlineMotions)
pool.close()
pool.join()

pc2 = time.perf_counter()
print("{} motions queried in {:0.2f} seconds".format(len(onlineMotions), pc2 - pc1))
print()

for motion in onlineMotions:
    motion.summarise()
    print()

4 motions queried in 2.93 seconds

motion-619

Username = 
Identifier = 619

Hardware = wireless-mini
Firmware = 3120

Battery = 78%
Battery is not charging

GNSS rate = 5


motion-621

Username = 
Identifier = 621

Hardware = wireless-mini
Firmware = 3120

Battery = 72%
Battery is not charging

GNSS rate = 5


motion-633

Username = 
Identifier = 633

Hardware = wireless-mini
Firmware = 3120

Battery = 76%
Battery is not charging

GNSS rate = 5


motion-634

Username = 
Identifier = 634

Hardware = wireless-mini
Firmware = 3120

Battery = 79%
Battery is not charging

GNSS rate = 5


